In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans

In [9]:
#supervised
# Defining data transformation
transform = transforms.Compose([transforms.ToTensor(),])

# Loading the MNIST training
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Loading the MNIST test 
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Creating data loaders
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Function to generate wrong labels
def generate_wrong_label(label, num_classes):
    wrong_label = (label + torch.randint(1, num_classes, (1,))) % num_classes
    return wrong_label

# Modifing the data by adding one-hot encoded labels and wrong labels for negative data
num_classes = 10

def generate_negative_data(data, labels, num_classes):
    batch_size = data.size(0)
    one_hot_labels = torch.zeros(batch_size, num_classes)
    one_hot_labels[torch.arange(batch_size), labels] = 1

    one_hot_labels = one_hot_labels.unsqueeze(2).unsqueeze(3).expand(-1, -1, data.size(2), data.size(3))

    wrong_labels = generate_wrong_label(labels, num_classes)

    data_with_labels = torch.cat((one_hot_labels, data), dim=1)

    negative_data_with_labels = torch.clone(data_with_labels)
    negative_data_with_labels[:, :num_classes] = 1 - negative_data_with_labels[:, :num_classes]
    negative_data_with_labels[:, wrong_labels] = 1 - negative_data_with_labels[:, wrong_labels]

    return data_with_labels, negative_data_with_labels

# use
for images, labels in train_loader:
    data_with_labels, negative_data_with_labels = generate_negative_data(images, labels, num_classes)
    break 
    
#the answer of quwstion: By using this loss function, it is expected that the model would be encouraged to correctly classify
#positive and negative examples by minimizing the difference between the predicted values and the threshold.
#The logarithmic transformation and averaging help capture the goodness or quality of the model's predictions by assigning 
#higher penalties to larger deviations and providing an overall measure of performance through the average loss.
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.fc = nn.Linear(num_features, 1)
    
    def forward(self, x):
        x = self.fc(x)
        return x
def custom_loss(output, target, threshold=0):
    loss = torch.mean(torch.log(1 + torch.exp((threshold - target) * output)))
    return 4 * loss

# Definig the model
num_features = 2 * 28 * 28
model = CustomModel(num_features)

# use
data_with_labels, negative_data_with_labels = generate_negative_data(images, labels, num_classes)
input_data = torch.cat((data_with_labels.view(-1, num_features), negative_data_with_labels.view(-1, num_features)), dim=0)
outputs = model(input_data)

batch_size = 64
target = torch.zeros((batch_size * 2, 1))

outputs = outputs.view(-1, 1)

outputs = outputs[:batch_size * 2]

loss = custom_loss(outputs, target)

total_loss = 0
total_correct = 0

model.eval()

for images, labels in test_loader:
    data_with_labels, negative_data_with_labels = generate_negative_data(images, labels, num_classes)
    input_data = torch.cat((data_with_labels.view(-1, num_features), negative_data_with_labels.view(-1, num_features)), dim=0)
    
    positive_target = labels.repeat(2) 
    negative_target = torch.zeros_like(labels).repeat(2) 
    target = torch.cat((positive_target, negative_target))  
    
    outputs = model(input_data)
    
    target = target[:outputs.shape[0]]
    
    loss = custom_loss(outputs, target) 
    
    total_loss += loss.item()
    _, predicted = outputs.max(1)
    total_correct += predicted[:len(labels)].eq(labels).sum().item()   
    
average_loss = total_loss / len(test_dataset)
accuracy = total_correct / len(test_dataset)
error = 1 - accuracy

print(f"Test Loss: {average_loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Error: {error * 100:.2f}%")

Test Loss: 0.0462
Test Accuracy: 9.80%
Test Error: 90.20%


In [7]:
#unsupervised
# generating the mask
def generate_mask(data):
    return 1 - data

# generating hybrid data using the mask
def generate_hybrid_data(data, mask):
    return data * mask

class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.fc = nn.Linear(num_features, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc(x))
        return x

# Custom loss function
def custom_loss(outputs, threshold):
    return torch.mean(torch.log(1 + torch.exp(threshold - outputs)) + torch.log(1 + torch.exp(outputs - threshold)))

# Training parameters
num_epochs = 10
learning_rate = 0.001
threshold = 0.5

# Assuming you have a training dataset, train_loader, with labeled data

# Initialize the model
num_features = 784
model = CustomModel(num_features)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for images, _ in train_loader:
        batch_size = images.size(0)

        positive_data = images.view(batch_size, -1)
        negative_data = generate_hybrid_data(positive_data, generate_mask(positive_data))

        hybrid_data = torch.cat((positive_data, negative_data), dim=0)

        optimizer.zero_grad()
        outputs = model(hybrid_data)
        loss = custom_loss(outputs, threshold)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

# Test the model
model.eval()

# Assuming you have a test dataset, test_loader, with labeled data

correct = 0
total = 0
i = 0

with torch.no_grad():

    for images, labels in test_loader:
        i+=1
        batch_size = images.size(0)
        inputs = images.view(batch_size, -1)
        outputs = model(inputs)

        predicted_labels = (outputs > threshold).float()
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()
        
average_loss = total_loss / len(test_dataset)
accuracy = total_correct / len(test_dataset)
error = 1 - accuracy

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test Error: {error * 100:.2f}%")

Epoch [1/10], Loss: 1.3866
Epoch [2/10], Loss: 1.3865
Epoch [3/10], Loss: 1.3865
Epoch [4/10], Loss: 1.3865
Epoch [5/10], Loss: 1.3865
Epoch [6/10], Loss: 1.3864
Epoch [7/10], Loss: 1.3864
Epoch [8/10], Loss: 1.3864
Epoch [9/10], Loss: 1.3864
Epoch [10/10], Loss: 1.3864
Test Accuracy: 9.80%
Test Error: 90.20%


In [8]:
#the answer in part3: the network output represents the predicted probability of the positive class, and it is related to the 
#labels through the training process and the custom loss function. The model aims to learn to produce output values that align 
#with the true labels, and the loss function provides a measure of the discrepancy between the predictions and the labels, 
#which is used to update the model's parameters.

#Part4:Linear classifiers are computationally efficient and relatively simple compared to more complex classifiers
#Linear classifiers are computationally efficient and relatively simple compared to more complex classifiers
#